# Analysis of Energy Use per Bus

In [61]:
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import glob
import json
%matplotlib inline

## Bus Power Model

In [62]:
class BusModel(object):
    g = 9.81 # m/s^2 (acceleration of free-fall due to gravity)
    p_a = 1.225 # (density of air) kg/m^3
    C_d = 0.7 # (drag coefficient)
    
    def __init__(self, M, A_f, P_tire, R_W, lambd=1, n_t=0.8, p_rot_in=0.1):
        self.M = M # Vehicle mass (kg)
        self.M_equ = M * p_rot_in # Rotation inertia (kg)
        self.A_f = A_f # Frontal area (m^2)
        self.P_tire = P_tire # Tire pressure (bars)
        self.R_W = R_W # Wheel radius (m)
        self.lambd = lambd # Final transmission efficiency * gear ratio
        self.n_t = n_t # Overrall transmission efficiency
    
    def P_M(self, V, dV_dT, V_wind=0, alpha=0):
        return self._T_M(V, dV_dT, V_wind, alpha) * self._omega_M(V) # kW
    
    def _omega_M(self, V):
        return V / self.R_W # radians/second
    
    def _T_M(self, V, dV_dT, V_wind, alpha):
        return (self.lambd/self.n_t) * self.R_W * (self._F_total(V, V_wind, alpha) + (self.M + self.M_equ) * dV_dT) # Nm
    
    def _F_total(self, V, V_wind, alpha=0):
        return self._F_a(V, V_wind) + self._F_r(V, alpha) + self._F_g(alpha) # N
    
    def _F_a(self, V, V_wind):
        return 0.5 * self.p_a * self.A_f * self.C_d * np.square(V - V_wind) # N
    
    def _F_r(self, V, alpha):
        return self._C_r(V) * self.M * self.g * np.cos(alpha) # N
    
    def _C_r(self, V):
        return 0.005 + (1 / self.P_tire) * (0.01 + 0.0095 * np.square(V * 3600 / 100000))
    
    def _F_g(self, alpha):
        return self.M * self.g * np.sin(alpha) # N

## Extract Kinematics and Energy Data

In [85]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a)) * 1000

def extract_kinematics_and_energy_data(model, csv_filepath):
    df = pd.read_csv(csv_filepath)
    df["Timestamp"] = df["Timestamp"].str.replace(".000-04:00", "").apply(lambda t: datetime.strptime(t, "%Y-%m-%dT%H:%M:%S"))
    bus_ids = df["ID"].unique()
    result = pd.DataFrame(columns=df.columns)
    for bus_id in bus_ids:
        bus_df = df[df["ID"] == bus_id]
        bus_df.insert(2, "Time difference (s)", bus_df["Timestamp"].diff().apply(lambda t: t.total_seconds()), True)
        bus_df.insert(9, "Traveled (m)", haversine(bus_df.Latitude.shift(), bus_df.Longitude.shift(), bus_df.loc[:, "Latitude"], bus_df.loc[:, "Longitude"]), True)
        bus_df.insert(10, "Total Traveled (km)", np.nancumsum(bus_df["Traveled (m)"]) / 1000, True) 
        bus_df.insert(11, "Speed (m/s)", bus_df.apply(lambda row: row["Traveled (m)"]/row["Time difference (s)"] if row["Time difference (s)"] > 0 else np.nan, axis=1), True)
        bus_df.insert(12, "Change in Speed (m/s)", bus_df["Speed (m/s)"].diff(), True)
        bus_df.insert(13, "Acceleration (m/s^2)", bus_df.apply(lambda row: row["Change in Speed (m/s)"]/row["Time difference (s)"] if row["Time difference (s)"] > 0 else np.nan, axis=1), True)
        bus_df.insert(14, "Instantaneous Power (W)", model.P_M(bus_df["Speed (m/s)"], bus_df["Acceleration (m/s^2)"]), True)
        bus_df.insert(15, "Instantaneous Energy (kWh)", bus_df["Instantaneous Power (W)"] * bus_df["Time difference (s)"] / 3.6e6, True)
        bus_df.insert(16, "Cumulative Energy (kWh)", np.nancumsum(bus_df["Instantaneous Energy (kWh)"]), True)
        result = pd.concat([result, bus_df])
    return result

## Model parameters

In [86]:
M = (42 * 80.7) + 13041 # kg
A_f = 2 * 3 # m^2
P_tire = 6.20528 # bar
R_W = 0.5715/2 # m
model = BusModel(M, A_f, P_tire, R_W)

## June 24 2020

In [87]:
june_24_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-06-24.csv")
june_24_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Total Traveled (km),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,4964,2020-06-24 05:01:28,M14A-SBS,HUDSON ST/W 12 ST,SELECT BUS WEST SIDE via 14 ST,262.61395,-74.005284,40.740057,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,0.000000
1,4964,2020-06-24 05:01:59,M14A-SBS,HUDSON ST/W 12 ST,SELECT BUS WEST SIDE via 14 ST,260.69006,-74.005574,40.738222,31.0,205.500323,0.20550,6.629043,NaN,NaN,NaN,NaN,0.000000
2,4964,2020-06-24 05:03:01,M14A-SBS,NaN,SELECT BUS WEST SIDE via 14 ST,261.3268,-74.005661,40.737660,62.0,62.919967,0.26842,1.014838,-5.614204,-0.090552,-720.459549,-0.012408,-0.012408
3,4964,2020-06-24 05:03:31,M14A-SBS,NaN,SELECT BUS WEST SIDE via 14 ST,261.3268,-74.005661,40.737660,30.0,0.000000,0.26842,0.000000,-1.014838,-0.033828,0.000000,0.000000,-0.012408
4,4964,2020-06-24 05:04:02,M14A-SBS,NaN,SELECT BUS WEST SIDE via 14 ST,261.3268,-74.005661,40.737660,31.0,0.000000,0.26842,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012408


In [67]:
fig = px.line(june_24_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

In [88]:
fig = px.line(june_24_kinematics_df, x="Timestamp", y="Total Traveled (km)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## June 25 2020

In [69]:
june_25_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-06-25.csv")
june_25_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,14,2020-06-25 06:22:53,M42,W 42 ST/7 AV,WEST SIDE PIER 12 AV CROSSTOWN,158.00887,-73.986850,40.755988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,14,2020-06-25 06:23:23,M42,W 42 ST/7 AV,WEST SIDE PIER 12 AV CROSSTOWN,156.96777,-73.987342,40.756196,30.0,47.458252,1.581942,NaN,NaN,NaN,NaN,0.000000
2,14,2020-06-25 06:23:53,M42,W 42 ST/8 AV,WEST SIDE PIER 12 AV CROSSTOWN,158.19860,-73.988405,40.756646,30.0,102.569065,3.418969,1.837027,0.061234,9428.591588,0.078572,0.078572
3,14,2020-06-25 06:24:24,M42,W 42 ST/8 AV,WEST SIDE PIER 12 AV CROSSTOWN,156.79166,-73.990051,40.757320,31.0,157.600466,5.083886,1.664917,0.053707,13415.649352,0.115524,0.194095
4,14,2020-06-25 06:25:25,M42,W 42 ST/9 AV,WEST SIDE PIER 12 AV CROSSTOWN,157.21759,-73.991272,40.757840,61.0,117.982250,1.934135,-3.149751,-0.051635,346.341395,0.005869,0.199964


In [70]:
fig = px.line(june_25_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## June 27 2020

In [71]:
june_27_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-06-27.csv")
june_27_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,13,2020-06-27 00:52:35,M42,12 AV/42 ST,EAST SIDE U N-1 AV CROSSTOWN,305.31122,-74.001396,40.762166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,13,2020-06-27 00:53:06,M42,12 AV/42 ST,EAST SIDE U N-1 AV CROSSTOWN,305.31122,-74.001396,40.762166,31.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
2,13,2020-06-27 00:53:37,M42,12 AV/42 ST,EAST SIDE U N-1 AV CROSSTOWN,305.31122,-74.001396,40.762166,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,2020-06-27 00:54:38,M42,12 AV/42 ST,EAST SIDE U N-1 AV CROSSTOWN,305.31122,-74.001396,40.762166,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13,2020-06-27 00:55:08,M42,12 AV/42 ST,EAST SIDE U N-1 AV CROSSTOWN,305.31122,-74.001396,40.762166,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
fig = px.line(june_27_kinematics_df[june_27_kinematics_df["ID"] != 13], x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## June 28 2020

In [73]:
june_28_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-06-28.csv")
june_28_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,13,2020-06-28 13:53:38,M42,W 42 ST/12 AV,EAST SIDE U N-1 AV CROSSTOWN,342.25534,-74.001235,40.762022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,13,2020-06-28 13:54:39,M42,W 42 ST/12 AV,EAST SIDE U N-1 AV CROSSTOWN,342.25534,-74.001235,40.762022,61.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
2,13,2020-06-28 13:55:09,M42,W 42 ST/12 AV,EAST SIDE U N-1 AV CROSSTOWN,342.25534,-74.001235,40.762022,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,2020-06-28 13:55:40,M42,W 42 ST/12 AV,EAST SIDE U N-1 AV CROSSTOWN,342.25534,-74.001235,40.762022,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13,2020-06-28 13:56:41,M42,W 42 ST/12 AV,EAST SIDE U N-1 AV CROSSTOWN,342.25534,-74.001235,40.762022,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
fig = px.line(june_28_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## June 29 2020

In [75]:
june_29_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-06-29.csv")
june_29_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,4958,2020-06-29 05:02:39,M14D-SBS,W 18 ST / 10 AV,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005867,40.744420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,4958,2020-06-29 05:03:40,M14D-SBS,W 18 ST / 10 AV,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005867,40.744420,61.0,0.00000,0.000000,NaN,NaN,NaN,NaN,0.000000
2,4958,2020-06-29 05:04:10,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,30.0,9.63234,0.321078,0.321078,0.010703,505.453893,0.004212,0.004212
3,4958,2020-06-29 05:05:11,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,61.0,0.00000,0.000000,-0.321078,-0.005264,0.000000,0.000000,0.004212
4,4958,2020-06-29 05:05:42,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,31.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004212


In [76]:
fig = px.line(june_29_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## June 30 2020

In [77]:
june_30_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-06-30.csv")
june_30_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,4962,2020-06-30 05:58:17,M14D-SBS,W 14 ST/7 AV,SELECT BUS LES DELANCEY-FDR via 14 ST,337.32733,-73.999260,40.738363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,4962,2020-06-30 05:58:48,M14D-SBS,W 14 ST/7 AV,SELECT BUS LES DELANCEY-FDR via 14 ST,337.32733,-73.999260,40.738363,31.0,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
2,4962,2020-06-30 05:59:49,M14D-SBS,E 14 ST/UNIVERSITY PL,SELECT BUS LES DELANCEY-FDR via 14 ST,337.10983,-73.995287,40.736696,61.0,382.633634,6.272683,6.272683,0.102831,23820.283103,0.403621,0.403621
3,4962,2020-06-30 06:00:20,M14D-SBS,E 14 ST/UNIVERSITY PL,SELECT BUS LES DELANCEY-FDR via 14 ST,337.74323,-73.992123,40.735380,31.0,304.104787,9.809832,3.537149,0.114102,41767.861238,0.359668,0.763289
4,4962,2020-06-30 06:00:50,M14D-SBS,E 14 ST/UNIVERSITY PL,SELECT BUS LES DELANCEY-FDR via 14 ST,337.74323,-73.991989,40.735325,30.0,12.840281,0.428009,-9.381822,-0.312727,-2453.493641,-0.020446,0.742843


In [78]:
fig = px.line(june_30_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## July 01 2020

In [79]:
july_01_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-07-01.csv")
july_01_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,4958,2020-07-01 05:01:15,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,4958,2020-07-01 05:01:45,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,30.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
2,4958,2020-07-01 05:02:16,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4958,2020-07-01 05:02:46,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4958,2020-07-01 05:03:48,M14D-SBS,9 AV/W 18 ST,SELECT BUS LES DELANCEY-FDR via 14 ST,337.00815,-74.005767,40.744378,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
fig = px.line(july_01_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## July 02 2020

In [81]:
july_02_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-07-02.csv")
july_02_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,4950,2020-07-02 05:38:37,M34A-SBS,9 AV/W 40 ST,SELECT BUS WATERSIDE via 34 ST via 2 AV,234.16235,-73.993777,40.756849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,4950,2020-07-02 05:39:07,M34A-SBS,W 34 ST/9 AV,SELECT BUS WATERSIDE via 34 ST via 2 AV,233.42697,-73.993912,40.756666,30.0,23.310204,0.777007,NaN,NaN,NaN,NaN,0.000000
2,4950,2020-07-02 05:39:37,M34A-SBS,W 34 ST/9 AV,SELECT BUS WATERSIDE via 34 ST via 2 AV,233.42697,-73.994914,40.755289,30.0,174.835345,5.827845,5.050838,0.168361,30645.445978,0.255379,0.255379
3,4950,2020-07-02 05:40:38,M34A-SBS,W 34 ST/8 AV,SELECT BUS WATERSIDE via 34 ST via 2 AV,336.97340,-73.995005,40.752833,61.0,273.202290,4.478726,-1.349119,-0.022117,4052.984331,0.068676,0.324054
4,4950,2020-07-02 05:41:09,M34A-SBS,W 34 ST/8 AV,SELECT BUS WATERSIDE via 34 ST via 2 AV,337.18668,-73.994282,40.752528,31.0,69.707553,2.248631,-2.230095,-0.071939,-618.068513,-0.005322,0.318732


In [82]:
fig = px.line(july_02_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)

## July 03 2020

In [83]:
july_03_kinematics_df = extract_kinematics_and_energy_data(model, "../../data/bustime_log_data/2020-07-03.csv")
july_03_kinematics_df.head()

,ID,Timestamp,Route,Next stop,Destination,Bearing,Longitude,Latitude,Time difference (s),Traveled (m),Speed (m/s),Change in Speed (m/s),Acceleration (m/s^2),Instantaneous Power (W),Instantaneous Energy (kWh),Cumulative Energy (kWh)
0,4958,2020-07-03 06:40:56,M86-SBS,W 86 ST/BROADWAY,SELECT BUS YORKVILLE 1AV-92 ST CROSSTOWN,337.47208,-73.976956,40.788498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,4958,2020-07-03 06:41:26,M86-SBS,W 86 ST/BROADWAY,SELECT BUS YORKVILLE 1AV-92 ST CROSSTOWN,337.47208,-73.977007,40.788519,30.0,4.887516,0.162917,NaN,NaN,NaN,NaN,0.000000
2,4958,2020-07-03 06:41:57,M86-SBS,W 86 ST/BROADWAY,SELECT BUS YORKVILLE 1AV-92 ST CROSSTOWN,337.47208,-73.976956,40.788498,31.0,4.887516,0.157662,-0.005255,-0.000170,209.427930,0.001803,0.001803
3,4958,2020-07-03 06:42:58,M86-SBS,W 86 ST/AMSTERDAM AV,SELECT BUS YORKVILLE 1AV-92 ST CROSSTOWN,337.56702,-73.975400,40.787843,61.0,149.883623,2.457109,2.299447,0.037696,5419.204714,0.091825,0.093629
4,4958,2020-07-03 06:43:28,M86-SBS,W 86 ST/COLUMBUS AV,SELECT BUS YORKVILLE 1AV-92 ST CROSSTOWN,336.95953,-73.974048,40.787273,30.0,130.281243,4.342708,1.885600,0.062853,12247.431225,0.102062,0.195691


In [84]:
fig = px.line(july_03_kinematics_df, x="Timestamp", y="Cumulative Energy (kWh)", title="Cumulative Energy Use", line_group="ID", color="ID", hover_name="ID")

fig.update_xaxes(rangeslider_visible=True)